## Library

In [22]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import ExtraTreeClassifier

## Load CSV

In [4]:
df = pd.read_csv('./datasets/balanced_har70.csv')
df

,timestamp,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z,label
0,2021-05-21 14:39:06.159,-0.887939,-0.148438,-0.628174,-0.994629,0.184570,-0.042480,1
1,2021-03-29 15:07:45.500,-1.577148,0.088867,-0.774902,0.114502,-0.053955,-0.212891,1
2,2021-03-29 15:06:30.260,-0.667969,-0.069336,-0.396484,-0.953613,-0.177490,-0.183594,1
3,2021-03-24 15:10:11.779,-1.383057,-0.228760,-0.405273,-1.863770,0.125488,0.687988,1
4,2021-04-16 14:53:05.704,-0.671387,0.096436,-0.241211,-0.869629,-0.064941,-0.102783,1
...,...,...,...,...,...,...,...,...
26077,2021-04-16 14:40:44.904,-0.385254,0.062256,0.922607,0.071777,0.054932,-1.078125,8
26078,2021-04-16 14:39:32.865,-0.400146,0.094482,0.904541,0.076416,0.099121,-1.073730,8
26079,2021-04-15 14:32:38.419,-0.467041,0.236084,0.861328,0.110840,-0.247803,-1.078125,8
26080,2021-05-21 14:28:52.139,-0.484375,0.078125,0.732178,0.148193,-0.069092,-0.976074,8


## Dataset Partition

In [9]:
datasets = (df.iloc[:,1:-1].values, df.iloc[:,-1].values)
X, y = datasets

## Hyperparameter's selection with Cross validation

### RandomForestClassifier

In [14]:
level_depth = np.arange(1, 25, 1)

clf = HalvingRandomSearchCV(RandomForestClassifier(), {'max_depth': level_depth},
cv=10, random_state=42)

clf.fit(X, y)
print(clf.best_estimator_)

c:\Python311\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 24 is smaller than n_iter=186. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomForestClassifier(max_depth=18)


### Kneighbors

In [17]:
neighbors = np.arange(1, 20, 1)

clf = HalvingRandomSearchCV(KNeighborsClassifier(), {'n_neighbors': neighbors}, 
cv=10, random_state=0)
clf.fit(X, y)
print(clf.best_estimator_)

c:\Python311\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 19 is smaller than n_iter=186. Running 19 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


KNeighborsClassifier(n_neighbors=4)


### ExtraTreeClassifier

In [20]:
level_depth = np.arange(1, 50, 1)

clf = HalvingRandomSearchCV(ExtraTreeClassifier(random_state=0), {'max_depth': level_depth},
cv=10, random_state=42)
clf.fit(X, y)
print(clf.best_estimator_)

c:\Python311\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 49 is smaller than n_iter=186. Running 49 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


ExtraTreeClassifier(max_depth=34, random_state=0)


###  BaggingClassifier

In [25]:
n_estimators = np.arange(1, 20, 1)

clf = HalvingRandomSearchCV(BaggingClassifier(), {'n_estimators': n_estimators},
cv=10, random_state=0)
clf.fit(X, y)
print(clf.best_estimator_)


c:\Python311\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 19 is smaller than n_iter=186. Running 19 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


BaggingClassifier(n_estimators=16)


### MLPClassifier

In [12]:
num_layers = np.arange(1, 10, 2)
num_neurons = np.arange(5, 50, 10)
num_neurons

layers = []
for l in num_layers:
    for n in num_neurons:
        layers.append(l*[n])
    
clf = HalvingRandomSearchCV(MLPClassifier(max_iter=10000), {'hidden_layer_sizes': layers},
cv = 10, random_state = 0)
clf.fit(X, y)
print(clf.best_estimator_)

c:\Python311\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 25 is smaller than n_iter=372. Running 25 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=[35], max_iter=10000)
